# Read the data

Right now the notebook is set to work with fake data. This can be changed once the pipeline works.

The data is stored as a Dict[person_id, Sequences] where Sequences is a Dict[year, survery_wave_response]

Tokenizing takes a bit of time

In [10]:
import pandas as pd
import numpy as np
import time

import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.utils.data import TensorDataset

from sklearn.model_selection import train_test_split

from model.rnn import GRUDecoder
from model.autoencoder import AutoEncoder

from data_processing.encoding.categorical import CategoricalTransformer
from data_processing.encoding.numeric_and_date import ToQuantileTransformer
from data_processing.encoding.text2vec import TextTransform
from data_processing.sequences.sequencing import to_sequences

In [11]:
# read in data and prepare transformations
data = pd.read_csv('data/other_data/PreFer_fake_data.csv')
targets = pd.read_csv('data/other_data/PreFer_fake_outcome.csv')
#data = pd.read_csv('data/training/PreFer_train_data.csv', nrows = 200)
#targets = pd.read_csv('data/training/PreFer_train_outcome.csv', nrows = 200)
codebook = pd.read_csv('data/codebooks/PreFer_codebook.csv')
summary = pd.read_csv('data/codebooks/PreFer_codebook_summary.csv')

categorical_columns = codebook[(codebook.var_name.str.startswith('c')) & (codebook.type_var == 'categorical')].var_name.tolist()
quantile_columns = codebook[(codebook.var_name.str.startswith('c')) & ((codebook.type_var == 'numeric') | (codebook.type_var == 'date or time'))].var_name.tolist()

cat_transform = CategoricalTransformer()
cat_transform.fit(codebook)

/Users/lmmi/fertility-prediction-challenge/data_processing/encoding/categorical.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  core_cat_df['values_cat'] = core_cat_df['values_cat'].str.split("; ").apply(lambda x: [e.strip() for e in x])
/Users/lmmi/fertility-prediction-challenge/data_processing/encoding/categorical.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  core_cat_df['labels_cat'] = core_cat_df['labels_cat'].str.split("; ").apply(lambda x: [e.strip() for e in x])


In [12]:
then = time.time()
# this cell takes a bit of time
for col in categorical_columns:
    data[col] = cat_transform.transform(data[col])
now = time.time()
print(now -then)

11.095808029174805


In [13]:
quantile_transform = ToQuantileTransformer(quantile_columns)

then = time.time()
quantile_transform.fit(data)
data = quantile_transform.transform(data)
now = time.time()
print(now - then)


data.fillna(101, inplace=True)
data[quantile_columns] = data[quantile_columns].astype(int)

data2 = data[data.columns[data.columns.str.startswith('c')]]
data2['nomem_encr'] = data['nomem_encr']

sequences = to_sequences(data2, summary)

64.69245409965515


/var/folders/y6/j9fbqcvx6lb5l99614n30y4c0000gn/T/ipykernel_2445/1961966370.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data2['nomem_encr'] = data['nomem_encr']
/var/folders/y6/j9fbqcvx6lb5l99614n30y4c0000gn/T/ipykernel_2445/1961966370.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['nomem_encr'] = data['nomem_encr']


In [14]:
def get_device():
    # Check if CUDA is available
    if torch.cuda.is_available():
        # If CUDA is available, select the first CUDA device
        device = torch.device("cuda:0")
        print("Using CUDA device:", torch.cuda.get_device_name(0))
    # Check for MPS availability on supported macOS devices (requires PyTorch 1.12 or newer)
    elif torch.backends.mps.is_available():
        # If MPS is available, use MPS device
        device = torch.device("mps")
        print("Using MPS (Metal Performance Shaders) device")
    else:
        # Fallback to CPU if neither CUDA nor MPS is available
        device = torch.device("cpu")
        print("Using CPU")
    return device
device = get_device()

Using MPS (Metal Performance Shaders) device


In [15]:
# There are some strings that have still not been properly been filtered out
# this cell gets rid of them. 
# The real solution is to change the categorical encoding class
for _, wave_responses in sequences.items():
    for year, wave_response in wave_responses.items():
        
        not_int = np.array([not isinstance(x, int) for x in wave_response], dtype = bool)
        wave_responses[year] = [
            item if not _bool else 101 for (item, _bool) in zip(wave_response, not_int)
        ]

# Train the autoencoder

In [60]:
year_seq = {}


In [16]:
import torch
from torch.utils.data import Dataset

class YearlySequencesDataset(Dataset):
    def __init__(self, data_dict):
        super().__init__()
        self.samples = {}
        for year_dict in data_dict.values():
            for year, sequence in year_dict.items():
                if year in year_seq:
                    self.samples[year].append(sequence)
                else:
                    self.samples[year] = [sequence]
        # Flatten the dictionary structure and store data
        
        for person_id, years_data in data_dict.items():
            for year, sequence in years_data.items():
                self.samples.append((person_id, year, sequence))
                
    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        person_id, year, sequence = self.samples[idx]
        sequence_tensor = torch.tensor(sequence, dtype=torch.float32)  # Convert sequence to tensor
        return person_id, year, sequence_tensor


SyntaxError: closing parenthesis ']' does not match opening parenthesis '{' on line 7 (842475307.py, line 9)

In [17]:
# We dont need targets or year information for the autoencoder
# so we merge everthing together in one tensor
autoencoder_data = torch.tensor([
                                wave_response
                                for _, wave_responses in sequences.items()
                                for _, wave_response in wave_responses.items()
                        ]).to(torch.int64)

In [18]:
### Initialization of the Autoencoder 
HIDDEN_DIM = 512
ENCODING_SIZE = 64
BATCH_SIZE = 64
num_epochs_autoencoder = 10
learning_rate_autoencoder = 0.001


first_sequence = next(iter(sequences.values()))  # Get the first sequence
first_survey_wave = next(iter(first_sequence.values()))  # Get the first survey wave from the first sequence
SEQ_LEN = len(first_survey_wave)

vocab_size= len(set([ elem for  _, sequence in sequences.items() for _, item in sequence.items() for elem in item]))

train_dataloader = DataLoader(autoencoder_data, batch_size=BATCH_SIZE, shuffle=True)
autoencoder = AutoEncoder(vocab_size=vocab_size, embedding_size=HIDDEN_DIM, encoding_size=ENCODING_SIZE, sequence_len=SEQ_LEN).to(device)

error = nn.MSELoss()
optimizer = optim.Adam( autoencoder.parameters())

/opt/anaconda3/envs/PreFer/lib/python3.12/site-packages/torch/nn/modules/lazy.py:181: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [19]:
autoencoder.train()
for epoch in range(num_epochs_autoencoder):
    for batch in train_dataloader:
        batch = batch.to(device)
        
        optimizer.zero_grad()
        xx = autoencoder(batch)
        loss = nn.functional.mse_loss(xx, autoencoder.embedding(batch))
        
        loss.backward()
        
        optimizer.step()
    print(f'epoch {epoch} \t Loss: {loss.item():.4g}')

epoch 0 	 Loss: 0.7067
epoch 1 	 Loss: 0.6533
epoch 2 	 Loss: 0.6463
epoch 3 	 Loss: 0.8106
epoch 4 	 Loss: 0.7555
epoch 5 	 Loss: 0.7296
epoch 6 	 Loss: 0.7343
epoch 7 	 Loss: 0.6999
epoch 8 	 Loss: 0.6891
epoch 9 	 Loss: 0.6726


# Train the RNN

First we need to create Dataset class that can hold both the target (stored in a pd.DataFrame) and the sequences.

The sequences will be of dimension 14 x encoding_dimension, because we have 14 years of surveys.

I have created some code for getting the data into the right format, but it might not be useful.

## Regarding masks
Right now the masking is done already in the encoding. I haven't found exactly where Mikkel implemented this.
So for now, assume that nothing is padded, and then we'll figure it out with Mikkel.

In [46]:
class SequencesWithTarget(Dataset):
    def __init__(self, sequences:dict, target: pd.DataFrame):
        self.sequences = sequences 
        self.target = targets.set_index(keys = 'nomem_encr').squeeze().to_dict()
        self.keys = list(sequences.keys())

    def __len__(self):
        return len(self.keys)

    def __getitem__(self, index):
        person_id = self.keys[index]
        
        target = self.target[person_id]
        sequence = self.sequences[person_id]

        return target, sequence

In [47]:
# its not everyone we have a target for, so we do restrict the data to 
# the ones with known outcomes

train_person_ids, test_person_ids = train_test_split(targets['nomem_encr'], test_size=0.2, random_state=42)

In [65]:
torch.BoolTensor([1,0,1])

tensor([ True, False,  True])

In [48]:
# structure the data as a Dict[person_id, survey_embedding_sequence] 
# where survey_embedding_sequence is a tensor of size 14 x embedding_dimension
rnn_data = {person_id:
                autoencoder.encode(
                    torch.tensor(
                        [ wave_response for _, wave_response in wave_responses.items()]
                    ).to(device)
                )
            for person_id, wave_responses in sequences.items()
           }

In [49]:
# split data based on the splits made for the target
train_data = {person_id: rnn_data[person_id] for person_id in train_person_ids}
test_data = {person_id: rnn_data[person_id] for person_id in test_person_ids}


In [24]:
train_dataset = SequencesWithTarget(train_data, target = targets)
test_dataset = SequencesWithTarget(test_data, target = targets)

rnn_batch_size = 64

train_dataloader = DataLoader(train_dataset, batch_size=rnn_batch_size, shuffle=True)
test_dataloader  = DataLoader(test_dataset,  batch_size=rnn_batch_size)

In [62]:
HIDDEN_SIZE = 10

num_epochs_rnn = 2
learning_rate_rnn = 0.001

rnn_model = GRUDecoder(
    input_size=ENCODING_SIZE,
    hidden_size=HIDDEN_SIZE,
    max_seq_len=14
).to(device)

# assume that all 14 years are observed for everyone
single_mask = torch.BoolTensor([True]*14).to(device) 

# Define loss function and optimizer for RNN
rnn_loss = torch.nn.BCELoss()
rnn_optimizer = torch.optim.Adam(rnn_model.parameters(), lr=learning_rate_rnn)
# Training loop

rnn_model.train()
for epoch in range(num_epochs_rnn):
    print(epoch)
    running_loss = 0

    for batch in train_dataloader:
        labels, inputs = batch
        labels, inputs = labels.to(torch.float).to(device), inputs.to(device)

        rnn_optimizer.zero_grad() 

        # Forward pass
        mask = torch.stack([single_mask]*len(labels) ) #  not correct masking

        xx = rnn_model(inputs, mask)
        outputs = torch.nn.functional.sigmoid(xx)

        loss = rnn_loss(torch.flatten(outputs), labels)  

        #loss.backward(retain_graph=True)
        loss.backward()
        rnn_optimizer.step()

        running_loss += loss.item() * inputs.size(0)

    # Calculate average loss for the epoch
    epoch_loss = running_loss / len(train_dataloader.dataset)

    print(f"Epoch {epoch+1}/{num_epochs_rnn}, Loss: {epoch_loss:.4f}")j
    

0


RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [51]:
import torch
from sklearn.metrics import precision_recall_fscore_support

def evaluate_model(model, dataloader, single_mask):
    model.eval()  # Set the model to evaluation mode
    predictions, actuals = [], []
    
    with torch.no_grad():
        for batch in dataloader:
            labels, inputs = batch
            labels, inputs = labels.to(torch.float).to(device), inputs.to(device)
            
            # Forward pass
            mask = torch.stack([single_mask]*len(labels))
            outputs = model(inputs, mask)
            outputs = torch.nn.functional.sigmoid(outputs)
            
            # Convert outputs to binary predictions
            predicted_labels = (outputs > 0.5).float()  # Threshold predictions
            print(labels)
            print(predicted_labels.flatten())
            print('')
            
            # Store predictions and actual labels
            predictions.append(predicted_labels.flatten())
            actuals.append(labels.flatten())
    
    # Concatenate all the batches
    predictions = torch.cat(predictions)
    actuals = torch.cat(actuals)
    
    # Calculate precision, recall, and F1 score
    precision, recall, f1, _ = precision_recall_fscore_support(actuals.cpu().numpy(), predictions.cpu().numpy(), average='binary')
    
    return precision, recall, f1


In [52]:
precision, recall, f1 = evaluate_model(rnn_model, test_dataloader, single_mask)
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

tensor([0., 0., 0., 1., 0., 1.], device='mps:0')
tensor([0., 0., 0., 0., 0., 0.], device='mps:0')

Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000


/opt/anaconda3/envs/PreFer/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
